In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import glob
import json
from pathlib import Path

In [2]:
def load_json(dir):
    try:
        with open(dir) as f:
            data = json.load(f)
            
        return data
    
    except:
        return None

In [3]:
RESULT_DIR = "..\\result_summary"

In [4]:
def parse_file_nm(file_nm):
    mcts_info = None
    
    components = file_nm.split("\\")
    
    if "debug" in components:
        components.remove("debug")
    
    method = components[2]
    
    diff_cut = 1
    
    if 'am' in method:
        _, _, method, _, prob_type, num_probs, model_info, baseline_info, _ = components
        
    elif 'mcts' in method:
        _, _, method, diff_cut, _, prob_type, num_probs, model_info, baseline_info, mcts_info, _ = components
        diff_cut = float(diff_cut.split("-")[-1])

    num_probs = num_probs.split("-")[0]
    num_probs = int(num_probs.split("_")[1])
    baseline_info = baseline_info.split("-")[-1]
    
    return method, prob_type, num_probs, model_info, baseline_info, mcts_info, diff_cut

In [5]:
for file in glob.glob(f"{RESULT_DIR}/**/*.json", recursive=True):
    result = load_json(file)

    avg_result = False
    for k, v in result.items():
        for k2, v2 in v.items():
            if 'result' in k2:
                avg_result = True
                break
    
    if not avg_result and len(result) != 102:
        print(len(result))
        print(file)
        
# need to do diff-0.75 and CVRP with N=50 for 4 and 6

In [6]:
# parse all json file named with "all_result_avg.json" from the RESULT_DIR

all_result = {}

for file in glob.glob(f"{RESULT_DIR}/**/all_result_avg.json", recursive=True):
    data = load_json(file)
    # print(file)
    method, prob_type, num_probs, model_info, baseline_info, mcts_info, diff_cut = parse_file_nm(file)
    
    if data is not None:
        
        if method not in all_result:
            all_result[method] = {}
        
        if prob_type not in all_result[method]:
            all_result[method][prob_type] = {}
            
        if num_probs not in all_result[method][prob_type]:
            all_result[method][prob_type][num_probs] = {}
        
        if mcts_info is None:
            if model_info not in all_result[method][prob_type][num_probs]:
                all_result[method][prob_type][num_probs][model_info] = {}
            
            if baseline_info not in all_result[method][prob_type][num_probs][model_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info] = data
                
        else:
            if model_info not in all_result[method][prob_type][num_probs]:
                all_result[method][prob_type][num_probs][model_info] = {}
                
            if baseline_info not in all_result[method][prob_type][num_probs][model_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info] = {}
                
            if mcts_info not in all_result[method][prob_type][num_probs][model_info][baseline_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info][mcts_info] = {}
                
            if diff_cut not in all_result[method][prob_type][num_probs][model_info][baseline_info][mcts_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info][mcts_info][diff_cut] = data
                
    # break

In [7]:
def get_parital_result(method, prob_type, num_prob):       
    partial_result = {'model_info': [], 'baseline': [], 'mcts_info': [], 'epoch': [], 'score': [], 'score_std': [], 'runtime': [],  'runtime_std': [], 'diff_cut': []}
    
    def _append_epoch_result(model_info, baseline_info, epoch, mcts, epoch_score, epoch_score_std, epoch_runtime, epoch_runtime_std, diff_cut):
        partial_result['model_info'].append(model_info)
        partial_result['baseline'].append(baseline_info)
        partial_result['mcts_info'].append(mcts)
        partial_result['epoch'].append(epoch)                
        partial_result['score'].append(epoch_score)
        partial_result['score_std'].append(epoch_score_std)
        partial_result['runtime'].append(epoch_runtime)
        partial_result['runtime_std'].append(epoch_runtime_std)
        partial_result['diff_cut'].append(diff_cut)

    target_result = all_result[method][prob_type][num_prob]
    
    if "am" in method:
        for model_info in target_result:
            for baseline_info in target_result[model_info]:
                all_epoch_results = target_result[model_info][baseline_info]
                
                for epoch in all_epoch_results:
                    current_epoch_result = all_epoch_results[epoch]['result_avg']
                    epoch_score = current_epoch_result['score']
                    epoch_score_std = all_epoch_results[epoch]['result_std']['score']
                    epoch_runtime = current_epoch_result['runtime']
                    epoch_runtime_std = all_epoch_results[epoch]['result_std']['runtime']
                    
                    _append_epoch_result(model_info, baseline_info, epoch, "am", epoch_score,epoch_score_std, epoch_runtime, epoch_runtime_std, 1)
                    
    elif 'mcts' in method:
        for model_info in target_result:
            for baseline_info in target_result[model_info]:
                all_mcts_results = target_result[model_info][baseline_info]
                
                for mcts_info in all_mcts_results:
                    all_diff_results = all_mcts_results[mcts_info]
                    
                    for diff_cut in all_diff_results:
                        all_epoch_results = all_diff_results[diff_cut]
                    
                        for epoch in all_epoch_results:
                            current_epoch_result = all_epoch_results[epoch]['result_avg']
                            epoch_score = current_epoch_result['score']
                            epoch_score_std = all_epoch_results[epoch]['result_std']['score']
                            epoch_runtime = current_epoch_result['runtime']
                            epoch_runtime_std = all_epoch_results[epoch]['result_std']['runtime']
                            
                            _append_epoch_result(model_info, baseline_info, epoch, mcts_info, epoch_score,epoch_score_std, epoch_runtime, epoch_runtime_std, diff_cut)                                         
            
    df = pd.DataFrame.from_dict(partial_result)
    df['activation'] = df['model_info'].apply(lambda x: x.split("-")[5])
    df['enc_layers'] = df['model_info'].apply(lambda x: x.split('-')[2])
    df = df.drop(columns=['model_info'])
    
    df['train_score'] = df.apply(lambda x: x['epoch'].split("-")[1].split("=")[1], axis=1).astype(float)
    df['epoch'] =  df.apply(lambda x: x['epoch'].split("-")[0].split("=")[1], axis=1).astype(int)
    
    if 'mcts' in method:
        df['cpuct'] = df['mcts_info'].apply(lambda x: x.split("-")[2].split('_')[1])
        df['ns'] = df['mcts_info'].apply(lambda x: x.split("-")[0].split('_')[1])
        df['cpuct'] = df['cpuct'].astype(float)
        df['ns'] = df['ns'].astype(int)
    
    elif 'am' in method:
        df['cpuct'] = 0
        df['ns'] = 0
        
    df = df.drop(columns=['mcts_info'])
    

        
    return df

In [8]:
def plot_bar_result(base_df, baseline, activation, prob_type, num_prob, plot_dev=False, hue='cpuct'):
    if prob_type == 'tsp':
        y_ranges = {20: (3.75, 3.95), 50: (5.7, 5.875), 100: (7.95, 8.25)}
        
    elif prob_type == 'cvrp':
        y_ranges = {20: (6.1, 7.5), 50: (9, 12), 100: (7.95, 8.25)}
        
    _df = base_df[(base_df['baseline'] == baseline) & (base_df['activation'] == activation)]
    
    # drop rows where the mcts_info is 0.8
    _df = _df[_df['cpuct'] != '0.8']
    
    # sort by the train score
    _df = _df.sort_values(by=['epoch', 'cpuct', 'ns']).reset_index(drop=True)
    _df['score_std'] = _df['score_std'].astype(float)
    
    _df['cpuct'] = _df['cpuct'].astype(object)
    _df['cpuct'] = _df['cpuct'].fillna('am')
    
    _df['ns'] = _df['ns'].astype(object)
    _df['ns'] = _df['ns'].fillna('am')
    
    plt.figure(figsize=(12, 7))
    
    ax = sns.barplot(data=_df, x=_df['epoch'], y='score', hue=hue)
    
    if plot_dev:
        bar_coords = [(rect.get_x(), rect.get_y(), rect.get_width(), rect.get_height()) for rect in ax.patches]
                
        for i, (x, y, w, h) in enumerate(bar_coords):
            x_pos = x + w / 2
            y_top = y + h + 0.01
            
            plt.errorbar(x=x_pos, y=y_top, yerr=_df['score_std'][i], fmt='none', color='black', capsize=4)
    
    plt.title(f"{prob_type}_{num_prob}_{baseline}_{activation}")
    plt.ylim(*y_ranges[num_prob])
    plt.legend()
    
    path = Path(f"../result_image/bars")
    
    if not path.exists():
        path.mkdir(parents=True)
        
    
    plt.savefig(f"{path}/{prob_type}_{num_prob}_{baseline}_{activation}.png")
    plt.show()

In [9]:
# tsp_20 = pd.concat([get_parital_result('am', 'tsp', 20), get_parital_result('mcts_v2', 'tsp', 20)])

# for _baseline in ['mean', 'val']:
#     for _activation in ['relu', 'swiglu']:
#         plot_bar_result(tsp_20, _baseline, _activation, 'tsp', 20, hue='cpuct')

In [10]:
tsp_100 = pd.concat([get_parital_result('am', 'tsp', 100), get_parital_result('mcts', 'tsp', 100)])
tsp_100.to_csv("../result_csv/tsp_100.csv")
tsp_100.sort_values(by=['score']).head(15)

,baseline,epoch,score,score_std,runtime,runtime_std,diff_cut,activation,enc_layers,train_score,cpuct,ns
499,val,299,7.952167,0.072568,10.248525,25.766765,0.75,swiglu,6,7.86777,1.1,1000
414,mean,299,7.952278,0.073115,1.087797,0.199681,0.75,swiglu,6,7.90623,1.1,100
527,val,299,7.952654,0.072702,0.581626,0.353638,0.50,swiglu,6,7.86777,1.1,500
497,val,299,7.952794,0.072653,0.929683,1.953379,0.50,swiglu,6,7.86777,1.1,1000
464,mean,299,7.952995,0.073776,4.318947,5.818327,0.75,swiglu,6,7.90623,1.1,500
514,val,299,7.953113,0.072455,0.438101,0.087440,0.50,swiglu,6,7.86777,1.1,250
531,val,249,7.953231,0.071119,5.765433,5.832914,0.75,swiglu,6,7.84949,1.1,500
450,mean,299,7.953557,0.073392,3.438926,2.169177,0.75,swiglu,6,7.90623,1.1,250
483,val,299,7.953631,0.074261,1.265813,0.224949,0.75,swiglu,6,7.86777,1.1,100
433,mean,299,7.953920,0.072868,8.395795,24.720814,0.75,swiglu,6,7.90623,1.1,1000


In [11]:
for env_type in ['tsp', 'cvrp']:
    for N in [20, 50, 100]:
        df = pd.concat([get_parital_result('am', env_type, N), get_parital_result('mcts', env_type, N)])
        df.to_csv(f"../result_csv/{env_type}_{N}.csv", index=False)

In [12]:
df

,baseline,epoch,score,score_std,runtime,runtime_std,diff_cut,activation,enc_layers,train_score,cpuct,ns
0,mean,49,16.717783,3.231875,0.471543,0.000779,1.00,relu,4,16.39185,0.0,0
1,mean,99,16.606976,3.071906,0.473988,0.001013,1.00,relu,4,16.41725,0.0,0
2,mean,149,16.574835,2.968623,0.479116,0.001992,1.00,relu,4,16.01232,0.0,0
3,mean,199,16.535594,3.113413,0.480392,0.001704,1.00,relu,4,16.35231,0.0,0
4,mean,249,16.490532,3.008314,0.476007,0.000952,1.00,relu,4,15.99893,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
346,val,199,16.474388,3.018032,11.897707,14.392978,0.75,swiglu,6,16.17082,1.1,500
347,val,149,16.473369,3.139008,12.321263,14.749940,0.75,swiglu,6,16.41855,1.1,500
348,val,299,16.347532,3.049330,10.432255,12.280734,0.75,swiglu,6,16.26320,1.1,500
349,val,249,16.437976,3.054953,12.052590,14.915801,0.75,swiglu,6,16.53491,1.1,500


In [45]:
# load tsp and N=20

def get_pivot_table(env_type, N, epoch=299, enc_layers=6, diff_cut=0.75, index=['epoch', 'ns', 'activation', 'baseline'], refine_to_str=True):
    df = pd.read_csv(f"../result_csv/{env_type}_{N}.csv")
    cond1 = df['epoch'] == epoch
    cond2 = df['enc_layers'] == enc_layers
    cond3 = df['diff_cut'] == diff_cut 
    cond4 = df['diff_cut'] == 1
    cond5 = df['ns'] != 250
    df = df[cond1 & cond2 & (cond3 | cond4) & cond5] 
    pivot_table = df.pivot_table(index=index, values=['score', 'score_std', 'runtime', 'runtime_std'])
    # # reindex the column order as to score mean, score std, runtime mean, runtime std
    pivot_table = pivot_table.reindex(columns=['score', 'score_std', 'runtime', 'runtime_std'])
    if refine_to_str:
        pivot_table['score'] = pivot_table.apply(lambda x: f"{round(x['score'], 5):.4f} ± {round(x['score_std'], 2):.2f}", axis=1)
        pivot_table = pivot_table.drop(['score_std'], axis=1)
    
        pivot_table['runtime'] = pivot_table.apply(lambda x: f"{round(x['runtime'], 4):.3f} ± {round(x['runtime_std'], 2):.2f}", axis=1)
        pivot_table = pivot_table.drop(['runtime_std'], axis=1)
        
    return pivot_table

In [46]:
tsp_results = [get_pivot_table('tsp', N, refine_to_str=False) for N in [20, 50, 100]]
all_results_df = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)
all_results_df


20                                   \
                                   score score_std   runtime runtime_std   
epoch ns   activation baseline                                             
299   0    relu       mean      3.849166  0.087340  0.106683    0.000164   
                      val       3.848943  0.084740  0.111188    0.000175   
           swiglu     mean      3.846409  0.084434  0.109879    0.000128   
                      val       3.848185  0.085138  0.112462    0.000178   
      100  relu       mean      3.848645  0.087409  0.098566    0.002793   
                      val       3.848350  0.084755  0.089544    0.001562   
           swiglu     mean      3.846195  0.084505  0.102594    0.004207   
                      val       3.847354  0.085008  0.108760    0.005548   
      500  relu       mean      3.849092  0.087306  0.143115    0.026448   
                      val       3.848350  0.084755  0.110884    0.014887   
           swiglu     mean      3.846063  0.084436  0.162155    0.043350   
                      val       3.847841  0.085045  0.168574    0.051304   
      1000 relu       mean      3.849092  0.087306  0.188020    0.081464   
                      val       3.848350  0.084755  0.129800    0.042372   
           swiglu     mean      3.846141  0.084478  0.225597    0.153658   
                      val       3.847841  0.085045  0.235934    0.161515   

                                     50                                   \
                                   score score_std   runtime runtime_std   
epoch ns   activation baseline                                             
299   0    relu       mean      5.736106  0.059724  0.231187    0.000463   
                      val       5.738576  0.063584  0.229614    0.000554   
           swiglu     mean      5.726738  0.069453  0.237449    0.000701   
                      val       5.740503  0.066726  0.233933    0.000357   
      100  relu       mean      5.737484  0.059985  0.372957    0.038249   
                      val       5.731046  0.065711  0.363083    0.039500   
           swiglu     mean      5.724850  0.068898  0.324971    0.032071   
                      val       5.737676  0.066269  0.356330    0.035140   
      500  relu       mean      5.734466  0.059488  1.187038    0.852000   
                      val       5.729078  0.065781  1.051615    0.785520   
           swiglu     mean      5.725983  0.069559  0.892633    0.612543   
                      val       5.737743  0.066359  1.108870    0.783436   
      1000 relu       mean      5.733857  0.059513  1.954974    2.788646   
                      val       5.730098  0.064709  1.772289    2.907688   
           swiglu     mean      5.725730  0.069605  1.506560    2.072723   
                      val       5.737280  0.066556  1.817081    2.383270   

                                     100                                   
                                   score score_std    runtime runtime_std  
epoch ns   activation baseline                                             
299   0    relu       mean      7.985253  0.078364   0.449912    0.000449  
                      val       8.048133  0.079650   0.450653    0.000271  
           swiglu     mean      7.956173  0.073098   0.454370    0.000451  
                      val       7.955108  0.072696   0.456002    0.000474  
      100  relu       mean      7.984709  0.078276   1.211698    0.225768  
                      val       8.046557  0.079659   1.284270    0.204340  
           swiglu     mean      7.952278  0.073115   1.087797    0.199681  
                      val       7.953631  0.074261   1.265813    0.224949  
      500  relu       mean      7.980796  0.078735   5.078510    6.875227  
                      val       8.066720  0.089803   5.402512    5.740922  
           swiglu     mean      7.952995  0.073776   4.318947    5.818327  
                      val       7.962960  0.071001   5.244361    6.108776  
      1

In [47]:
tsp_results = [get_pivot_table('tsp', N) for N in [20, 50, 100]]
all_results_df = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)
all_results_df


20                           50   \
                                        score       runtime          score   
epoch ns   activation baseline                                               
299   0    relu       mean      3.8492 ± 0.09  0.107 ± 0.00  5.7361 ± 0.06   
                      val       3.8489 ± 0.08  0.111 ± 0.00  5.7386 ± 0.06   
           swiglu     mean      3.8464 ± 0.08  0.110 ± 0.00  5.7267 ± 0.07   
                      val       3.8482 ± 0.09  0.113 ± 0.00  5.7405 ± 0.07   
      100  relu       mean      3.8486 ± 0.09  0.099 ± 0.00  5.7375 ± 0.06   
                      val       3.8483 ± 0.08  0.089 ± 0.00  5.7310 ± 0.07   
           swiglu     mean      3.8462 ± 0.08  0.103 ± 0.00  5.7248 ± 0.07   
                      val       3.8474 ± 0.09  0.109 ± 0.01  5.7377 ± 0.07   
      500  relu       mean      3.8491 ± 0.09  0.143 ± 0.03  5.7345 ± 0.06   
                      val       3.8483 ± 0.08  0.111 ± 0.01  5.7291 ± 0.07   
           swiglu     mean      3.8461 ± 0.08  0.162 ± 0.04  5.7260 ± 0.07   
                      val       3.8478 ± 0.09  0.169 ± 0.05  5.7377 ± 0.07   
      1000 relu       mean      3.8491 ± 0.09  0.188 ± 0.08  5.7339 ± 0.06   
                      val       3.8483 ± 0.08  0.130 ± 0.04  5.7301 ± 0.06   
           swiglu     mean      3.8461 ± 0.08  0.226 ± 0.15  5.7257 ± 0.07   
                      val       3.8478 ± 0.09  0.236 ± 0.16  5.7373 ± 0.07   

                                                        100                  
                                     runtime          score         runtime  
epoch ns   activation baseline                                               
299   0    relu       mean      0.231 ± 0.00  7.9852 ± 0.08    0.450 ± 0.00  
                      val       0.230 ± 0.00  8.0481 ± 0.08    0.451 ± 0.00  
           swiglu     mean      0.237 ± 0.00  7.9562 ± 0.07    0.454 ± 0.00  
                      val       0.234 ± 0.00  7.9551 ± 0.07    0.456 ± 0.00  
      100  relu       mean      0.373 ± 0.04  7.9847 ± 0.08    1.212 ± 0.23  
                      val       0.363 ± 0.04  8.0466 ± 0.08    1.284 ± 0.20  
           swiglu     mean      0.325 ± 0.03  7.9523 ± 0.07    1.088 ± 0.20  
                      val       0.356 ± 0.04  7.9536 ± 0.07    1.266 ± 0.22  
      500  relu       mean      1.187 ± 0.85  7.9808 ± 0.08    5.079 ± 6.88  
                      val       1.052 ± 0.79  8.0667 ± 0.09    5.402 ± 5.74  
           swiglu     mean      0.893 ± 0.61  7.9530 ± 0.07    4.319 ± 5.82  
                      val       1.109 ± 0.78  7.9630 ± 0.07    5.244 ± 6.11  
      1000 relu       mean      1.955 ± 2.79  7.9802 ± 0.08   9.986 ± 28.99  
                      val       1.772 ± 2.91  8.0565 ± 0.08  10.391 ± 22.82  
           swiglu     mean      1.507 ± 2.07  7.9539 ± 0.07   8.396 ± 24.72  
                      val       1.817 ± 2.38  7.9522 ± 0.07  10.248 ± 25.77

In [48]:
cvrp_results = [get_pivot_table('cvrp', N) for N in [20, 50, 100]]
all_results_df = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)
all_results_df


20                            50   \
                                        score       runtime           score   
epoch ns   activation baseline                                                
299   0    relu       mean      6.4097 ± 0.75  0.131 ± 0.00  10.8050 ± 1.65   
                      val       6.4553 ± 0.80  0.132 ± 0.00  10.8634 ± 1.65   
           swiglu     mean      6.4231 ± 0.73  0.133 ± 0.00  10.7940 ± 1.70   
                      val       6.4201 ± 0.72  0.132 ± 0.00  10.8402 ± 1.48   
      100  relu       mean      6.4010 ± 0.76  0.358 ± 0.07  10.8225 ± 1.59   
                      val       6.4452 ± 0.83  0.439 ± 0.09  10.8718 ± 1.61   
           swiglu     mean      6.4228 ± 0.75  0.320 ± 0.05  10.7747 ± 1.71   
                      val       6.4020 ± 0.74  0.398 ± 0.06  10.8420 ± 1.53   
      500  relu       mean      6.4065 ± 0.75  0.948 ± 0.69  10.8192 ± 1.60   
                      val       6.4449 ± 0.84  1.147 ± 0.79  10.8651 ± 1.65   
           swiglu     mean      6.4230 ± 0.73  0.820 ± 0.40  10.7878 ± 1.72   
                      val       6.4055 ± 0.73  0.983 ± 0.55  10.8336 ± 1.49   
      1000 relu       mean      6.4073 ± 0.76  1.514 ± 2.07  10.8087 ± 1.63   
                      val       6.4479 ± 0.80  1.830 ± 2.19  10.8774 ± 1.67   
           swiglu     mean      6.4292 ± 0.74  1.269 ± 1.08  10.7847 ± 1.67   
                      val       6.4101 ± 0.72  1.542 ± 1.54  10.8456 ± 1.53   

                                                          100                  
                                      runtime           score         runtime  
epoch ns   activation baseline                                                 
299   0    relu       mean       0.279 ± 0.00  16.4418 ± 2.99    0.523 ± 0.00  
                      val        0.276 ± 0.00  16.4463 ± 3.00    0.525 ± 0.00  
           swiglu     mean       0.278 ± 0.00  16.4043 ± 3.09    0.527 ± 0.00  
                      val        0.277 ± 0.00  16.3575 ± 3.04    0.525 ± 0.00  
      100  relu       mean       0.965 ± 0.15  16.4628 ± 3.00    2.550 ± 0.43  
                      val        0.934 ± 0.16  16.4750 ± 3.08    2.697 ± 0.47  
           swiglu     mean       0.921 ± 0.20  16.4190 ± 3.10    2.380 ± 0.55  
                      val        1.060 ± 0.19  16.3954 ± 2.97    2.253 ± 0.44  
      500  relu       mean       3.577 ± 3.44  16.4596 ± 2.99  12.464 ± 14.39  
                      val        3.686 ± 3.46  16.4494 ± 3.02  13.216 ± 11.13  
           swiglu     mean       3.562 ± 4.31  16.4170 ± 3.10  11.078 ± 15.41  
                      val        4.015 ± 4.37  16.3475 ± 3.05  10.432 ± 12.28  
      1000 relu       mean      6.211 ± 11.21  16.4387 ± 2.96  24.966 ± 63.28  
                      val       6.657 ± 14.16  16.4468 ± 2.98  26.552 ± 47.20  
           swiglu     mean      6.185 ± 14.44  16.4117 ± 3.11  22.032 ± 69.04  
                      val       7.070 ± 15.20  16.3492 ± 3.02  20.679 ± 46.96

In [49]:
tsp_results = [get_pivot_table('tsp', N, index=['activation']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['activation']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                            50                 \
                         score       runtime           score       runtime   
     activation                                                              
tsp  relu        3.8487 ± 0.09  0.122 ± 0.02   5.7338 ± 0.06  0.895 ± 0.93   
     swiglu      3.8470 ± 0.08  0.153 ± 0.05   5.7321 ± 0.07  0.810 ± 0.74   
cvrp relu        6.4272 ± 0.79  0.812 ± 0.74  10.8416 ± 1.63  2.823 ± 4.07   
     swiglu      6.4170 ± 0.73  0.700 ± 0.46  10.8128 ± 1.60  2.921 ± 4.84   

                            100                  
                          score         runtime  
     activation                                  
tsp  relu         8.0186 ± 0.08    4.282 ± 8.11  
     swiglu       7.9549 ± 0.07    3.934 ± 7.86  
cvrp relu        16.4526 ± 3.00  10.437 ± 17.11  
     swiglu      16.3877 ± 3.06   8.738 ± 18.09

In [50]:
tsp_results = [get_pivot_table('tsp', N, index=['activation'], refine_to_str=False) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['activation'], refine_to_str=False) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                                         50   \
                    score score_std   runtime runtime_std      score   
     activation                                                        
tsp  relu        3.848748  0.086046  0.122225    0.021233   5.733839   
     swiglu      3.847004  0.084761  0.153244    0.052486   5.732063   
cvrp relu        6.427232  0.786270  0.812225    0.738553  10.841632   
     swiglu      6.416989  0.734850  0.699662    0.460143  10.812817   

                                                       100            \
                score_std   runtime runtime_std      score score_std   
     activation                                                        
tsp  relu        0.062312  0.895345    0.926577   8.018611  0.080933   
     swiglu      0.067928  0.809728    0.740030   7.954904  0.072923   
cvrp relu        1.630561  2.823111    4.073124  16.452558  3.001521   
     swiglu      1.603551  2.921023    4.837910  16.387702  3.059856   

                                        
                   runtime runtime_std  
     activation                         
tsp  relu         4.281748    8.107637  
     swiglu       3.933951    7.855030  
cvrp relu        10.436604   17.113115  
     swiglu       8.738353   18.085536

In [51]:
tsp_results = [get_pivot_table('tsp', N, index=['baseline']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['baseline']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                            50                 \
                       score       runtime           score       runtime   
     baseline                                                              
tsp  mean      3.8476 ± 0.09  0.142 ± 0.04   5.7306 ± 0.06  0.839 ± 0.80   
     val       3.8481 ± 0.08  0.133 ± 0.03   5.7352 ± 0.07  0.867 ± 0.87   
cvrp mean      6.4153 ± 0.75  0.686 ± 0.55  10.7996 ± 1.66  2.747 ± 4.22   
     val       6.4289 ± 0.77  0.826 ± 0.65  10.8549 ± 1.57  2.997 ± 4.69   

                          100                 
                        score        runtime  
     baseline                                 
tsp  mean       7.9683 ± 0.08   3.873 ± 8.35  
     val        8.0052 ± 0.08   4.343 ± 7.61  
cvrp mean      16.4319 ± 3.04  9.565 ± 20.39  
     val       16.4084 ± 3.02  9.610 ± 14.81

In [52]:
tsp_results = [get_pivot_table('tsp', N, index=['activation', 'baseline']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['activation', 'baseline']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                            50   \
                                  score       runtime           score   
     activation baseline                                                
tsp  relu       mean      3.8490 ± 0.09  0.134 ± 0.03   5.7355 ± 0.06   
                val       3.8485 ± 0.08  0.110 ± 0.01   5.7322 ± 0.06   
     swiglu     mean      3.8462 ± 0.08  0.150 ± 0.05   5.7258 ± 0.07   
                val       3.8478 ± 0.09  0.156 ± 0.05   5.7383 ± 0.07   
cvrp relu       mean      6.4061 ± 0.76  0.738 ± 0.71  10.8138 ± 1.62   
                val       6.4483 ± 0.82  0.887 ± 0.77  10.8694 ± 1.64   
     swiglu     mean      6.4245 ± 0.74  0.635 ± 0.38  10.7853 ± 1.70   
                val       6.4094 ± 0.73  0.764 ± 0.54  10.8404 ± 1.51   

                                                   100                  
                               runtime           score         runtime  
     activation baseline                                                
tsp  relu       mean      0.936 ± 0.92   7.9827 ± 0.08    4.181 ± 9.02  
                val       0.854 ± 0.93   8.0545 ± 0.08    4.382 ± 7.19  
     swiglu     mean      0.740 ± 0.68   7.9538 ± 0.07    3.564 ± 7.68  
                val       0.879 ± 0.80   7.9560 ± 0.07    4.304 ± 8.03  
cvrp relu       mean      2.758 ± 3.70  16.4507 ± 2.99  10.126 ± 19.53  
                val       2.888 ± 4.45  16.4544 ± 3.02  10.747 ± 14.70  
     swiglu     mean      2.736 ± 4.74  16.4130 ± 3.10   9.004 ± 21.25  
                val       3.106 ± 4.94  16.3624 ± 3.02   8.473 ± 14.92

In [53]:
tsp_results = [get_pivot_table('tsp', N, index=['ns', 'activation']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['ns', 'activation']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                            50   \
                              score       runtime           score   
     ns   activation                                                
tsp  0    relu        3.8491 ± 0.09  0.109 ± 0.00   5.7373 ± 0.06   
          swiglu      3.8473 ± 0.08  0.111 ± 0.00   5.7336 ± 0.07   
     100  relu        3.8485 ± 0.09  0.094 ± 0.00   5.7343 ± 0.06   
          swiglu      3.8468 ± 0.08  0.106 ± 0.00   5.7313 ± 0.07   
     500  relu        3.8487 ± 0.09  0.127 ± 0.02   5.7318 ± 0.06   
          swiglu      3.8470 ± 0.08  0.165 ± 0.05   5.7319 ± 0.07   
     1000 relu        3.8487 ± 0.09  0.159 ± 0.06   5.7320 ± 0.06   
          swiglu      3.8470 ± 0.08  0.231 ± 0.16   5.7315 ± 0.07   
cvrp 0    relu        6.4325 ± 0.78  0.131 ± 0.00  10.8342 ± 1.65   
          swiglu      6.4216 ± 0.73  0.133 ± 0.00  10.8171 ± 1.59   
     100  relu        6.4231 ± 0.79  0.398 ± 0.08  10.8471 ± 1.60   
          swiglu      6.4124 ± 0.75  0.359 ± 0.05  10.8084 ± 1.62   
     500  relu        6.4257 ± 0.79  1.048 ± 0.74  10.8421 ± 1.62   
          swiglu      6.4143 ± 0.73  0.902 ± 0.47  10.8107 ± 1.60   
     1000 relu        6.4276 ± 0.78  1.672 ± 2.13  10.8430 ± 1.65   
          swiglu      6.4196 ± 0.73  1.405 ± 1.31  10.8152 ± 1.60   

                                                100                  
                            runtime           score         runtime  
     ns   activation                                                 
tsp  0    relu         0.230 ± 0.00   8.0167 ± 0.08    0.450 ± 0.00  
          swiglu       0.236 ± 0.00   7.9556 ± 0.07    0.455 ± 0.00  
     100  relu         0.368 ± 0.04   8.0156 ± 0.08    1.248 ± 0.22  
          swiglu       0.341 ± 0.03   7.9530 ± 0.07    1.177 ± 0.21  
     500  relu         1.119 ± 0.82   8.0238 ± 0.08    5.240 ± 6.31  
          swiglu       1.001 ± 0.70   7.9580 ± 0.07    4.782 ± 5.96  
     1000 relu         1.864 ± 2.85   8.0184 ± 0.08  10.188 ± 25.91  
          swiglu       1.662 ± 2.23   7.9530 ± 0.07   9.322 ± 25.24  
cvrp 0    relu         0.277 ± 0.00  16.4441 ± 2.99    0.524 ± 0.00  
          swiglu       0.278 ± 0.00  16.3809 ± 3.06    0.526 ± 0.00  
     100  relu         0.950 ± 0.15  16.4689 ± 3.04    2.623 ± 0.45  
          swiglu       0.991 ± 0.19  16.4072 ± 3.04    2.316 ± 0.50  
     500  relu         3.631 ± 3.45  16.4545 ± 3.01  12.840 ± 12.76  
          swiglu       3.788 ± 4.34  16.3823 ± 3.07  10.755 ± 13.84  
     1000 relu        6.434 ± 12.68  16.4428 ± 2.97  25.759 ± 55.24  
          swiglu      6.627 ± 14.82  16.3805 ± 3.06  21.355 ± 58.00

In [59]:
tsp_results = [get_pivot_table('tsp', N, index=['ns', 'activation'], refine_to_str=False) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)
tsp_results = tsp_results.reset_index()
ns_col_name = tsp_results.columns[0]
ns_col_name

('ns', '')

In [60]:
for env_type in ['tsp', 'cvrp']:
    for N in [20, 50, 100]:
        corr_coef = get_pivot_table(env_type, N, index=['ns'], refine_to_str=False).reset_index().corr().loc['ns', 'score']
        print(f"{env_type}_{N}: {corr_coef}")

tsp_20: -0.21013197915874848
tsp_50: -0.7522634277815382
tsp_100: 0.1934597929508303
cvrp_20: 0.004099167909689254
cvrp_50: 0.7313441200081879
cvrp_100: -0.4519241133515795


In [61]:
def corr(ns, score):
    return np.corrcoef(ns, score)[0, 1]

for N in [20, 50, 100]:
    ns = tsp_results.loc[:, (ns_col_name, '')].values.flatten()
    score = tsp_results.loc[:, (N, 'score')]
    print(f"N={N}: ", corr(ns, score))


N=20:  -0.045468057282767295
N=50:  -0.5894648349451826
N=100:  0.01495662636221181


C:\Users\admin\AppData\Local\Temp\ipykernel_16648\1406123094.py:5: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  ns = tsp_results.loc[:, (ns_col_name, '')].values.flatten()
C:\Users\admin\AppData\Local\Temp\ipykernel_16648\1406123094.py:5: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  ns = tsp_results.loc[:, (ns_col_name, '')].values.flatten()
C:\Users\admin\AppData\Local\Temp\ipykernel_16648\1406123094.py:5: FutureWarning: The behavi

In [62]:
tsp_results = [get_pivot_table('tsp', N, index=['ns'], refine_to_str=False) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['ns'], refine_to_str=False) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                                         50             \
              score score_std   runtime runtime_std      score score_std   
     ns                                                                    
tsp  0     3.848176  0.085413  0.110053    0.000161   5.735481  0.064872   
     100   3.847636  0.085419  0.099866    0.003528   5.732764  0.065215   
     500   3.847836  0.085385  0.146182    0.033997   5.731818  0.065297   
     1000  3.847856  0.085396  0.194838    0.109752   5.731741  0.065096   
cvrp 0     6.427053  0.750962  0.131821    0.000285  10.825631  1.619546   
     100   6.417769  0.770504  0.378759    0.065744  10.827754  1.609033   
     500   6.419995  0.763661  0.974611    0.608843  10.826414  1.613908   
     1000  6.423625  0.757113  1.538583    1.722520  10.829100  1.625737   

                                       100                                   
            runtime runtime_std      score score_std    runtime runtime_std  
     ns                                                                      
tsp  0     0.233046    0.000519   7.986167  0.075952   0.452734    0.000411  
     100   0.354335    0.036240   7.984294  0.076328   1.212394    0.213684  
     500   1.060039    0.758375   7.990868  0.078329   5.011083    6.135813  
     1000  1.762726    2.538082   7.985701  0.077103   9.755187   25.575425  
cvrp 0     0.277441    0.000999  16.412478  3.028981   0.525150    0.001209  
     100   0.970084    0.173049  16.438038  3.039231   2.469875    0.472423  
     500   3.709948    3.896115  16.418374  3.039899  11.797689   13.303202  
     1000  6.530796   13.751903  16.411631  3.014643  23.557199   56.620467

In [63]:
temp = all_results_df.reset_index().rename(columns={'level_0': 'env_type'})
# reset multiindex of columns
def _get_new_col_name(col):
    if col[1] == '':
        return col[0]
    else:
        return f"{col[0]}_{col[1]}"
temp.columns = temp.columns.map(_get_new_col_name)
temp

,env_type,ns,20_score,20_score_std,20_runtime,20_runtime_std,50_score,50_score_std,50_runtime,50_runtime_std,100_score,100_score_std,100_runtime,100_runtime_std
0,tsp,0,3.848176,0.085413,0.110053,0.000161,5.735481,0.064872,0.233046,0.000519,7.986167,0.075952,0.452734,0.000411
1,tsp,100,3.847636,0.085419,0.099866,0.003528,5.732764,0.065215,0.354335,0.036240,7.984294,0.076328,1.212394,0.213684
2,tsp,500,3.847836,0.085385,0.146182,0.033997,5.731818,0.065297,1.060039,0.758375,7.990868,0.078329,5.011083,6.135813
3,tsp,1000,3.847856,0.085396,0.194838,0.109752,5.731741,0.065096,1.762726,2.538082,7.985701,0.077103,9.755187,25.575425
4,cvrp,0,6.427053,0.750962,0.131821,0.000285,10.825631,1.619546,0.277441,0.000999,16.412478,3.028981,0.525150,0.001209
5,cvrp,100,6.417769,0.770504,0.378759,0.065744,10.827754,1.609033,0.970084,0.173049,16.438038,3.039231,2.469875,0.472423
6,cvrp,500,6.419995,0.763661,0.974611,0.608843,10.826414,1.613908,3.709948,3.896115,16.418374,3.039899,11.797689,13.303202
7,cvrp,1000,6.423625,0.757113,1.538583,1.722520,10.829100,1.625737,6.530796,13.751903,16.411631,3.014643,23.557199,56.620467


In [65]:
temp_tsp = temp[temp['env_type'] == 'tsp']
temp_cvrp = temp[temp['env_type'] == 'cvrp']

tsp_mean = temp_tsp[temp_tsp['ns'] != 0].mean()
score_dict_tsp  = {k: v for k, v in tsp_mean.items() if 'score' in k}
score_dict_tsp

C:\Users\admin\AppData\Local\Temp\ipykernel_16648\3533312857.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  tsp_mean = temp_tsp[temp_tsp['ns'] != 0].mean()


{'20_score': 3.847776066660881,
 '20_score_std': 0.08540012451658675,
 '50_score': 5.732107611894608,
 '50_score_std': 0.06520260136349994,
 '100_score': 7.986954247951508,
 '100_score_std': 0.07725310717313898}

In [69]:
temp_tsp[temp_tsp['ns'] == 0][score_dict_tsp.keys()].to_dict(orient='records')[0]

{'20_score': 3.8481759107112885,
 '20_score_std': 0.08541296520352035,
 '50_score': 5.735480684041977,
 '50_score_std': 0.06487214975981208,
 '100_score': 7.98616651058197,
 '100_score_std': 0.07595200115354869}

In [66]:
cvrp_mean = temp_cvrp[temp_cvrp['ns'] != 0].mean()
score_dict_cvrp = {k: v for k, v in cvrp_mean.items() if 'score' in k}
score_dict_cvrp

C:\Users\admin\AppData\Local\Temp\ipykernel_16648\1176079181.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cvrp_mean = temp_cvrp[temp_cvrp['ns'] != 0].mean()


{'20_score': 6.420463389158248,
 '20_score_std': 0.7637593662607708,
 '50_score': 10.827756220499673,
 '50_score_std': 1.616225938724427,
 '100_score': 16.422680872281394,
 '100_score_std': 3.03125747720228}

In [70]:
temp_cvrp[temp_cvrp['ns'] == 0][score_dict_cvrp.keys()].to_dict(orient='records')[0]

{'20_score': 6.427052675485611,
 '20_score_std': 0.7509616997868286,
 '50_score': 10.825630614757538,
 '50_score_std': 1.6195458422344464,
 '100_score': 16.41247809886932,
 '100_score_std': 3.0289812984564497}